### TensorBoard

In [15]:
%load_ext tensorboard

SyntaxError: invalid syntax (1312197558.py, line 3)

In [ ]:
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
import numpy as np

In [13]:
# Load MNIST (pre-existing train and test sets)
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Concatenate the training and test sets (we want to split the training data into training, validation and test sets with a 70:20:10 ratio)
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)
# Split the validation data into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=1/3)

# Define the model architecture
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),  # Flatten the input
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),    # First hidden layer
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),                   # Dropout layer
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2') # Output layer
  ])

In [18]:
!rm -rf ./logs/

In [19]:
# Create and compile the model
model = create_model()
model.compile(optimizer='adam', # Use adaptive momentum estimation for adaptive gradient algorithm
              loss='sparse_categorical_crossentropy', # Use cross-entropy loss function for classification
              metrics=['accuracy']) # Use accuracy as the metric

# Setup TensorBoard logging
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with the training set and validate with the validation set
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

# Start TensorBoard within the notebook

!rm -rf ./logs/
%tensorboard --logdir logs/fit
!rm -rf ./logs/

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

# Log the test metrics to TensorBoard
with tf.summary.create_file_writer(log_dir).as_default():
  tf.summary.scalar('test_loss', test_loss, step=1)
  tf.summary.scalar('test_accuracy', test_acc, step=1)

Epoch 1/5
1313/1313 [==============================] - 2s 1ms/step - loss: 0.2582 - accuracy: 0.9237 - val_loss: 0.1403 - val_accuracy: 0.9597
Epoch 2/5
1313/1313 [==============================] - 2s 1ms/step - loss: 0.1124 - accuracy: 0.9652 - val_loss: 0.1069 - val_accuracy: 0.9697
Epoch 3/5
1313/1313 [==============================] - 2s 1ms/step - loss: 0.0798 - accuracy: 0.9745 - val_loss: 0.0957 - val_accuracy: 0.9730
Epoch 4/5
1313/1313 [==============================] - 2s 1ms/step - loss: 0.0599 - accuracy: 0.9812 - val_loss: 0.0867 - val_accuracy: 0.9735
Epoch 5/5
1313/1313 [==============================] - 2s 1ms/step - loss: 0.0466 - accuracy: 0.9851 - val_loss: 0.0863 - val_accuracy: 0.9772


Reusing TensorBoard on port 6006 (pid 46058), started 0:47:29 ago. (Use '!kill 46058' to kill it.)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2420 - accuracy: 0.9295 - val_loss: 0.1149 - val_accuracy: 0.9664

Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1056 - accuracy: 0.9682 - val_loss: 0.0855 - val_accuracy: 0.9736

Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0730 - accuracy: 0.9774 - val_loss: 0.0879 - val_accuracy: 0.9704

Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0557 - accuracy: 0.9824 - val_loss: 0.0717 - val_accuracy: 0.9773

Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0452 - accuracy: 0.9855 - val_loss: 0.0656 - val_accuracy: 0.9786


In [23]:
from tensorflow.keras.models import load_model

# Initialize results and models list
scores = []
models = []
fold_no = 0

# Loop through the indices the split() method returns
for train, test in kfold.split(x, y):
    fold_no += 1
    # Generate a print
    print(f'Training for fold {fold_no} ...')

    # Create and compile the model
    model = create_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Fit data to model
    model.fit(x[train], y[train], epochs=5)

    # Save the model
    model.save(f'model_{fold_no}.h5')
    models.append(f'model_{fold_no}.h5')

    # Generate generalization metrics
    scores.append(model.evaluate(x[test], y[test], verbose=0))

# == Provide average scores ==
print(f'Score per fold: {scores}')
print('------------------------------------------------------------------------')
print(f'> Accuracy: {np.mean(scores)} (+- {np.std(scores)})')
print('------------------------------------------------------------------------')

# Load and evaluate each saved model
for model_file in models:
    model = load_model(model_file)
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f'{model_file}: {score}')

Training for fold 1 ...
Epoch 1/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.2238 - accuracy: 0.9344
Epoch 2/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0982 - accuracy: 0.9696
Epoch 3/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0708 - accuracy: 0.9780
Epoch 4/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0550 - accuracy: 0.9819
Epoch 5/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0444 - accuracy: 0.9856
Training for fold 2 ...
Epoch 1/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.2234 - accuracy: 0.9334
Epoch 2/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0977 - accuracy: 0.9695
Epoch 3/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0696 - accuracy: 0.9781
Epoch 4/5
1750/1750 [==============================] - 2s 1ms/step - loss: 0.0532 - accuracy: 0.9834
Epoch 5/5
1750/1750 [======================